<a href="https://colab.research.google.com/github/LeoFernanndes/notebooks/blob/master/housing_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [0]:
import pandas as pd


url = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_train.csv'
df = pd.read_csv(url)

In [0]:
housing = df.copy()

# starting by filling the blank spaces 
for column in housing.columns:
    if housing[column].dtype != 'object':
        housing[column].fillna(housing[column].mean(), inplace= True)
    else:
        housing[column] = housing[column].astype('str')
        housing[column].fillna(housing[column].mode(), inplace= True)

In [221]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


features = ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition']



x_opt = housing[features]
y_opt = housing['SalePrice']


le_opt = LabelEncoder()
for column in x_opt.columns:
    if x_opt[column].dtype == 'object':
        x_opt[column] = le_opt.fit_transform(x_opt[column])

# x_opt_train, x_opt_test, y_opt_train, y_opt_test = train_test_split(x_opt, y_opt, test_size= 0.25)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [222]:
import numpy as np


df = x_opt
df['saleprice'] = y_opt

df2 = df.copy().iloc[1200:,:]
df = df.iloc[:1200,:]

array = df.values
X = array[:,0: len(df.columns) - 1]
Y = array[:, len(df.columns) - 1]


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression


# Feature extraction
test = SelectKBest(score_func= f_regression, k=len(df.columns) - 1)
fit = test.fit(X, Y)

# Summarize scores
np.set_printoptions(precision=3)
features = fit.transform(X)


chi2_list = []
for chi2 in fit.scores_:
  chi2_list.append(chi2)

features_list = df.columns[0: len(df.columns) - 1]

chi2_df = pd.DataFrame({'feature': features_list,
                        'chi2': chi2_list}).sort_values(by= ['chi2'], ascending= False).reset_index().drop(['index'], axis= 1)

chi2_features = list(chi2_df['feature'])
important_chi2_features = chi2_features[:3]

chi2_df


,feature,chi2
0,OverallQual,1977.578160
1,GrLivArea,1453.389408
2,TotalBsmtSF,839.612342
3,GarageCars,835.373548
4,ExterQual,806.257993
...,...,...
75,BsmtCond,0.279468
76,GarageQual,0.158777
77,BsmtFinType2,0.121598
78,Condition2,0.064265


In [223]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor


used_features = []
score = 1

for i in range(len(df.columns) - 1):

  

  if i == 0:
    used_features = []
    score = 1
  else:
    pass

  n = i + 1
  
  df_sorted = df[list(chi2_df['feature'])]
  array_sorted = df_sorted.values

  X_opt = array_sorted[:, 0: n]
  Y_opt = Y

  x_opt_train, x_opt_test, y_opt_train, y_opt_test = train_test_split(x_opt, y_opt, test_size= 0.25)

  reg_opt_it = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
 
  
  reg_opt_it.fit(x_opt_train, y_opt_train)
  y_opt_pred_it = reg_opt_it.predict(x_opt_test)

  erro_opt_it = np.sqrt(mean_squared_log_error(y_opt_test, y_opt_pred_it))
  erro_opt_it


  if erro_opt_it < score:
    score = erro_opt_it
    used_features.append(df_sorted.columns[i])

    print(score, df_sorted.columns[i])
  else:
    print('pass', erro_opt_it, score)

print('Number of used features: ', len(used_features), '\n',
    'Used feadtures:', '\n',
    used_features, '\n',
    'Score: ', score)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:55:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.014926993301439742 OverallQual
[15:55:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.007835687214216808 GrLivArea
[15:55:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

pass 0.014617232397709063 0.005278297170235739
[15:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


pass 0.027458146972029675 0.005278297170235739
[15:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


pass 0.018571192075078527 0.005278297170235739
Number of used features:  7 
 Used feadtures: 
 ['OverallQual', 'GrLivArea', 'BsmtQual', 'GarageFinish', 'Foundation', 'BsmtExposure', 'HalfBath'] 
 Score:  0.005278297170235739


In [224]:
reg_opt_it_treinado = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

reg_opt_it_treinado.fit(x_opt_train[used_features], y_opt_train)


[15:56:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
# making the predictions over submission set
# loading submission data
url_data = 'https://raw.githubusercontent.com/LeoFernanndes/datasets/master/housing_test.csv'
data = pd.read_csv(url_data)

In [0]:
# filling in the gaps in the same manner as used in the trein set
for column in data.columns:
    if data[column].dtype != 'object':
        data[column].fillna(data[column].mean(), inplace= True)
    else:
        data[column] = data[column].astype('str')
        data[column].fillna(data[column].mode(), inplace= True)

# using label encoder over the test set the same way as used in trein set 
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le_opt.fit_transform(data[column])

In [0]:
# df for kaggle evaluation
final = reg_opt_it_treinado.predict(data[used_features])
envio = pd.DataFrame({'Id': data['Id'], 'SalePrice': final})

In [0]:

# saving the df in a csv to be uploaded

#path = r'C:\Users\Avell\Desktop\Python\github\datasets'
#nome = '\housing_submission_2.csv'
#envio.to_csv(path+nome, index= False)


In [228]:
## calling a new xgboost w/o feature engineering

# mean rmsle from 10 different test splits

df = df[used_features]

n = 5
erro1 = 0
for i in(range(n)):
  
  x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:-1]], df[df.columns[-1]], test_size= 0.2)

  reg = RandomForestRegressor()

  reg.fit(x_train, y_train)
  y_pred = reg.predict(x_test)

  erro = np.sqrt(mean_squared_log_error(y_test, y_pred))
  print(erro)
  erro1 += erro

erro1/n

0.3154308574700915
0.31229042237400334
0.3217494162208188
0.30504680915281895
0.3060927164409201


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

0.3121220443317305

In [229]:
y_test_novo = df2[used_features].iloc[:,-1]
x_test_novo = df2[used_features].iloc[:, 0:-1]

y_pred_2 = reg.predict(x_test_novo)

erro = np.sqrt(mean_squared_log_error(y_test_novo, y_pred_2))
erro


0.3041056378555631